This code is based on an existing version (and slightly adapted towards our specific needs) here: https://www.pinecone.io/learn/llama-2/


In [ ]:
# Some necessary requirenements

!pip install -qU \
    transformers==4.31.0 \
    accelerate==0.21.0 \
    einops==0.6.1 \
    xformers==0.0.20 \
    bitsandbytes==0.41.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
import transformers

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)


In [ ]:
model_id = 'meta-llama/Llama-2-7b-chat-hf'

# begin initializing HF items, need auth token for these
hf_auth = '<YOUR-AUTHENTIFICATION TOKEN>'
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

In [ ]:
# initialize the model
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
    use_auth_token=hf_auth
)
model.eval()


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2193: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )


In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1714: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
generate_text = transformers.pipeline(
    model=model, tokenizer=tokenizer,
    #return_full_text=True,  # langchain expects the full text
    task='text-generation',
    # we pass model parameters here too
    temperature=0.9,  # 'randomness' of outputs, 0.1 is the min and 1.0 the max
    #max_new_tokens=512,  # mex number of tokens to generate in the output
    #repetition_penalty=1.1  # without this output begins repeating
)



In [ ]:
from google.colab import drive

# Mounting Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import re
def parse_classification_output(output_text):
    """
    INPUT: the generated Text by the model. This includes the given Prompt + Model answer as one string.

    FUNCTION:
    - Seperates model Prompt/Input and Model output by the ending tag of the Instruction-tag [\INST]
    - Matching the Summary part of the model, with a regular expression, extracting all text after the "Summary:" marker.
    - Matching the Class, with a second regular expression, extracting the first string after "Class:" that is part of the predefined classes (case insensitive)

    OUTPUT: A dictionary containing original prompt/model input (prompt), the complete model answer (model_answer), The extracted class (class), and the summary (summary)
    If no Class or Summary could be detected, a default string is entered instead.
    """


    # Split the output by the prompt and the model's answer
    prompt, model_answer = output_text.split('[/INST]')
    # Process 'model_answer' or to extract 'class'
    classes = [
    "FAMILY HISTORY/SOCIAL HISTORY", "HISTORY of PRESENT ILLNESS", "PAST MEDICAL HISTORY",
    "CHIEF COMPLAINT", "PAST SURGICAL HISTORY", "Allergy", "REVIEW OF SYSTEMS",
    "Medications", "Assessment", "Exam", "Diagnosis", "Disposition", "Plan",
    "EMERGENCY DEPARTMENT COURSE", "Immunizations", "Imaging", "GYNECOLOGIC HISTORY",
    "Procedures", "Other history", "Labs"
    ]
    #matching the Summary part
    pattern = r'Summary:\s*([\s\S]+?)(?=$)'
    match = re.search(pattern, model_answer)

    # If no match is found, a default string will raplace the summary:
    if match:
      summary_content = match.group(1).strip()
      cleaned_summary = re.sub(r'Class:.*?\n(\n\s*)+', '', summary_content, flags=re.MULTILINE)
    else:
      cleaned_summary = "Summary not found."
      #print("Summary not found.")

    # Process 'model_answer' to extract 'class':
    # Create a regex pattern that matches any of the classes, defined above:
    regex_pattern = r'\b(?:' + '|'.join(re.escape(cls) for cls in classes) + r')\b'

    # Find all matches in the text
    matches = re.findall(regex_pattern, model_answer, re.IGNORECASE)  # Use re.IGNORECASE for case-insensitive matching

    # Extract the top choice selected, in calse there were multiple matches for class (we assume the first one is the correct one)
    if len(matches) > 0:
        return {
            'prompt': prompt.strip(),
            'model_answer': model_answer.strip(),
            'class': matches[0].strip(),
            'summary': cleaned_summary.strip()
        }
    else:
        return {
            'prompt': prompt.strip(),
            'model_answer': model_answer.strip(),
            'class': 'No match found',  # Return an empty string if no match is found
            'summary': cleaned_summary.strip()
        }


In [ ]:
import csv
import os
import traceback


def process_conversation(system_prompt, input_conversation, shot):
    """
    INPUT:
    - system_prompt: a string with a personalized system prompt for the model
    - input_conversation: a string of one of the dialogues from the test set
    - shot: a string containing the example shots"""

    #data_1 = f"""<s>[INST]<<SYS>>\n{system_prompt}\n<</SYS>>\n\nCONTEXT:\n{shot}\nINPUT: {input_conversation}[/INST]"""

    #data_1_new = f"""<s>[INST]<<SYS>>\n{system_prompt}\n<</SYS>>\n\n{shot}{input_conversation}\n[/INST]"""

    data_2 = f"""<s>[INST]<<SYS>>\n{system_prompt}\n{shot}\n<</SYS>>\n\n{input_conversation}\n[/INST]"""


    torch.manual_seed(42)
    sequences = generate_text(
        data_2, # here replace the data template to the one you want to use
        do_sample=True, # Temperature on = True, Temperature off = False
        top_k=5,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=4000,
        )

    for seq in sequences:
      parsed_output = parse_classification_output(seq['generated_text'])
      # Example: Process conversation (echoes the same data)
      return parsed_output

#The filenames for input file (test data), output file (result file) and checkpoint tracker file:
input_file_name = '/content/drive/My Drive/NLP_Medicine_GC/MTS-Dialog-TestSet-1-MEDIQA-Chat-2023.csv'
output_file_name = '/content/drive/My Drive/NLP_Medicine_GC/D2_Class&Sum_folder/out_data_new_T09_SP2_CSum_2shot.csv'
checkpoint_file = '/content/drive/My Drive/NLP_Medicine_GC/D2_Class&Sum_folder/checkpoint.txt'

# Function to resume from checkpoint
def resume_from_checkpoint():
    index = 0  # Default index to start from

    # Check if the checkpoint file exists and read the last successful index
    if os.path.exists(checkpoint_file):
        with open(checkpoint_file, 'r') as file:
            index = int(file.read())
            print(f"Resuming from iteration {index}")

    return index
start_index = resume_from_checkpoint()

try:
    # Open input and output files to continue processing
    with open(input_file_name, mode='r', encoding='utf-8') as input_file, open(output_file_name, mode='a', encoding='utf-8', newline='') as output_file:
        reader = csv.DictReader(input_file)
        fieldnames = ['ID', 'prompt', 'model_answer', 'class', 'summary', 'original_index']
        writer = csv.DictWriter(output_file, fieldnames=fieldnames)

        # Check if the output file is empty, then write headers
        if output_file.tell() == 0:
            writer.writeheader()

        # Move to the starting index
        for _ in range(start_index):
            next(reader)

        for index, row in enumerate(reader, start=start_index):
            # Process conversation for each row
            #----Classification only task:
            #system_prompt_V1 = """Classify the conversation into one of these 20 classes: FAMILY HISTORY/SOCIAL HISTORY, HISTORY of PRESENT ILLNESS, PAST MEDICAL HISTORY, CHIEF COMPLAINT, PAST SURGICAL HISTORY, Allergy, REVIEW OF SYSTEMS, Medications, Assessment, Exam, Diagnosis, Disposition, Plan, EMERGENCY DEPARTMENT COURSE, Immunizations, Imaging, GYNECOLOGIC HISTORY, Procedures, Other history, Labs."""
            #system_prompt_V1_idx = """"Classify the input conversation into one of these 20 classes: 1 - FAMILY HISTORY/SOCIAL HISTORY, 2 - HISTORY of PRESENT ILLNESS, 3 - PAST MEDICAL HISTORY, 4 - CHIEF COMPLAINT, 5 - PAST SURGICAL HISTORY, 6 - Allergy, 7 - REVIEW OF SYSTEMS, 8 - Medications, 9 - Assessment, 10 - Exam, 11 - Diagnosis, 12 - Disposition, 13 - Plan, 14 - EMERGENCY DEPARTMENT COURSE, 15 - Immunizations, 16 - Imaging, 17 - GYNECOLOGIC HISTORY, 18 - Procedures, 19 - Other history, 20 - Labs."""
            #system_prompt_V2_idx = """Classify the conversation into one of these 20 classes: 1 - FAMILY HISTORY/SOCIAL HISTORY, 2 - HISTORY of PRESENT ILLNESS, 3 - PAST MEDICAL HISTORY, 4 - CHIEF COMPLAINT, 5 - PAST SURGICAL HISTORY, 6 - Allergy, 7 - REVIEW OF SYSTEMS, 8 - Medications, 9 - Assessment, 10 - Exam, 11 - Diagnosis, 12 - Disposition, 13 - Plan, 14 - EMERGENCY DEPARTMENT COURSE, 15 - Immunizations, 16 - Imaging, 17 - GYNECOLOGIC HISTORY, 18 - Procedures, 19 - Other history, 20 - Labs. Please provide specific keywords, phrases, or details from the conversation that indicate which class it belongs to. Always choose the most obvious class."""

            #-----Classification & Summarization Task:
            #system_prompt_SP1 = """"Classify the conversation into one of these 20 classes: 1 - FAMILY HISTORY/SOCIAL HISTORY, 2 - HISTORY of PRESENT ILLNESS, 3 - PAST MEDICAL HISTORY, 4 - CHIEF COMPLAINT, 5 - PAST SURGICAL HISTORY, 6 - Allergy, 7 - REVIEW OF SYSTEMS, 8 - Medications, 9 - Assessment, 10 - Exam, 11 - Diagnosis, 12 - Disposition, 13 - Plan, 14 - EMERGENCY DEPARTMENT COURSE, 15 - Immunizations, 16 - Imaging, 17 - GYNECOLOGIC HISTORY, 18 - Procedures, 19 - Other history, 20 - Labs. The response should start with the selected class, followed by the summary "Summary:" of the conversation in a clinical note style."""
            system_prompt_SP2 = """Classify the conversation into one of these 20 classes: FAMILY HISTORY/SOCIAL HISTORY, HISTORY of PRESENT ILLNESS, PAST MEDICAL HISTORY, CHIEF COMPLAINT, PAST SURGICAL HISTORY, Allergy, REVIEW OF SYSTEMS, Medications, Assessment, Exam, Diagnosis, Disposition, Plan, EMERGENCY DEPARTMENT COURSE, Immunizations, Imaging, GYNECOLOGIC HISTORY, Procedures, Other history, Labs. The response should start with the selected class, followed by the summary "Summary:" of the conversation in a clinical note style."""

            #-----Shots:
            #zero_shot = "The conversation is:"
            #The shots were taken from the validation set from the MST-Dialogues Data Collection:
            #val. set, ID 6 and #val. set, ID 3
            #one_shot = """EXAMPLE INPUT: Doctor: Have you had any surgeries in the past?\nPatient: Nope I have not.\nDoctor: Anything?\nPatient: No.\n\nClass: 5 - PAST SURGICAL HISTORY\n\nFollowing the same format above from the example input, determine the class of this conversation:""".strip()
            #one_shot_sum ="""EXAMPLE INPUT: Doctor: Have you had any surgeries in the past?\nPatient: Nope I have not.\nDoctor: Anything?\nPatient: No.\n\nClass: 5 - PAST SURGICAL HISTORY\n\nSummary: He has not had any previous surgery.\n\nFollowing the same format above from the example input, return the class and the summary of this conversation:""".strip()

            #two_shot = """EXAMPLE INPUT: Doctor: Have you had any surgeries in the past?\nPatient: Nope I have not.\nDoctor: Anything?\nPatient: No.\n\nClass: 5 - PAST SURGICAL HISTORY\n\nEXAMPLE INPUT: Doctor: How've you been treating your acne?\nPatient: The dermatologist started me on Accutane.\nDoctor: How long have you been on Accutane?\nPatient: About three months now.\nDoctor: Have you noticed any improvements since starting it?\nPatient: Yeah, kinda.\n\nClass: 8 - Medications\n\nFollowing the same format above from the example inputs, what is the class of this conversation:""".strip()
            #two_shot_adapted = """EXAMPLE INPUT: Doctor: Have you had any surgeries in the past?\nPatient: Nope I have not.\nDoctor: Anything?\nPatient: No.\n\nClass: 5 - PAST SURGICAL HISTORY\n\nEXAMPLE INPUT: Doctor: How've you been treating your acne?\nPatient: The dermatologist started me on Accutane.\nDoctor: How long have you been on Accutane?\nPatient: About three months now.\nDoctor: Have you noticed any improvements since starting it?\nPatient: Yeah, kinda.\n\nClass: 8 - Medications\n\nFollowing the same format from the above examples, what is the class of this conversation:""".strip()

            two_shot_sum = """EXAMPLE INPUT: Doctor: Have you had any surgeries in the past?\nPatient: Nope I have not.\nDoctor: Anything?\nPatient: No.\n\nClass: 5 - PAST SURGICAL HISTORY\n\nSummary: He has not had any previous surgery.\n\nEXAMPLE INPUT: Doctor: How've you been treating your acne?\nPatient: The dermatologist started me on Accutane.\nDoctor: How long have you been on Accutane?\nPatient: About three months now.\nDoctor: Have you noticed any improvements since starting it?\nPatient: Yeah, kinda.\n\nClass: 8 - Medications\n\nSummary: Accutane.\n\nFollowing the same format from the above examples, return the class and the summary of this conversation:""".strip()

            processed_conversation = process_conversation(system_prompt_SP2, row['dialogue'].strip(), two_shot_sum)  # Process using your model

            # Write to output file
            writer.writerow({
                'ID': row['ID'],
                'prompt': processed_conversation["prompt"],
                'model_answer': processed_conversation["model_answer"],
                'class': processed_conversation["class"],
                'summary': processed_conversation["summary"],
                'original_index': index
            })

            # Update the checkpoint file after every successful iteration
            with open(checkpoint_file, 'w') as checkpoint:
                checkpoint.write(str(index))

except Exception as e:
    print(f"Error occurred at iteration {index}: {e}")
    traceback.print_exc()  # Print detailed error information for debugging


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
